In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# A Simple Starter Code for the AJL Competition

@Cindy Deng

---


Hi! This starter code is designed to help you get familiar with basic Kaggle operations and guide you through the basic workflow of a machine learning project.

The code outlines essential steps including data loading, preprocessing, model building, training, and generating predictions. Each section serves as a foundation, but there are many ways to enhance each step to improve your final model's accuracy. Feel free to experiment with different data augmentation techniques, model architectures, and tuning methods to optimize your final results! Some amazing tutorials are available through your AI Studio course in Canvas / in the 'Resource' section of this Kaggle competition.

Good luck and have fun!

---

## Note - About file path

You could use the cell above to print the names of the file directories and get the following directories:

```
/kaggle/input/bttai-ajl-2025/sample_submission.csv
/kaggle/input/bttai-ajl-2025/train.csv
/kaggle/input/bttai-ajl-2025/test.csv
/kaggle/input/bttai-ajl-2025/test/test/e0374ae6c1362ff183cfba28ded5421b.jpg
/kaggle/input/bttai-ajl-2025/test/test/437159c605260bdd079af230566af291.jpg
...
...
/kaggle/input/bttai-ajl-2025/train/train/dermatomyositis/11271bdf2598afdd4260db3125e1f6a5.jpg
/kaggle/input/bttai-ajl-2025/train/train/dermatomyositis/732819951dcf2b53d15ea7b8bb123b71.jpg
/kaggle/input/bttai-ajl-2025/train/train/dermatomyositis/6dcc7a8abb5e1c6e670101f4b6231246.jpg
/kaggle/input/bttai-ajl-2025/train/train/dermatomyositis/e63c3b3f0ab8905e204fe467cc7411f9.jpg
...
...
```



## 1. Import Necessary Libraries

In [ ]:
# 1. Import Necessary Libraries
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

#Data augmentation:
import random
from tqdm import tqdm
from PIL import Image, ImageEnhance, ImageFilter
# Explanation:
# - pandas and numpy: for data manipulation
# - sklearn: for splitting data and encoding labels
# - tensorflow.keras: for building and training the neural network

## 2. Load Data

Make sure to verify the file paths if you're running on a different platform.

In [ ]:
# 2. Load Data
train_df = pd.read_csv('/kaggle/input/bttai-ajl-2025/train.csv')
test_df = pd.read_csv('/kaggle/input/bttai-ajl-2025/test.csv')

# Add .jpg extension to md5hash column to reference the file_name
train_df['md5hash'] = train_df['md5hash'].astype(str) + '.jpg'
test_df['md5hash'] = test_df['md5hash'].astype(str) + '.jpg'

# Combine label and md5hash to form the correct path
train_df['file_path'] = train_df['label'] + '/' + train_df['md5hash']

In [ ]:
# Check the first few rows to understand the structure
print(train_df.head())

In [ ]:
train_df['label'].nunique() #amount of unique labels

In [ ]:
#counting the amount of each skin tone on the FST assigned by centur lab
size = train_df.shape[0]
print(size)
#fitzpatrick_centaur rating
print(train_df["fitzpatrick_centaur"].nunique())
print(train_df["fitzpatrick_centaur"].unique())
#fitzpatrick_scale rating
print(train_df["fitzpatrick_scale"].nunique())
print(train_df["fitzpatrick_scale"].unique())

In [ ]:
print((train_df["fitzpatrick_centaur"]!=train_df["fitzpatrick_scale"]).sum())
mismatch = (train_df["fitzpatrick_centaur"]!=train_df["fitzpatrick_scale"]).sum()
print("percent of discrepencies between self perscribed skin shade and centur lab perscribes skine shade:", mismatch/size * 100,"%")

In [ ]:
# phase -1 in fitzpatrick_centaur
neg_one = (train_df["fitzpatrick_centaur"]==-1).sum()
print("amount of -1:",neg_one)
print("percentage of -1:", neg_one/size * 100,"%")

In [ ]:
#there are some elements that are not labeled in the centaur label but are in the self determined scale
#could I create a new one and try and populate this???
print(((train_df["fitzpatrick_centaur"]==-1) & ( train_df["fitzpatrick_scale"]!=-1)).sum())

In [ ]:
# phase 1 in fitzpatrick_centaur
one = (train_df["fitzpatrick_centaur"]==1).sum()
print("amount of 1:",one)
print("percentage of 1:", one/size * 100,"%")

In [ ]:
# phase 2 in fitzpatrick_centaur
two = (train_df["fitzpatrick_centaur"]==2).sum()
print("amount of 2:",two)
print("percentage of 2:", two/size * 100,"%")
print("additions 2 need to match 1:", one - two)

In [ ]:
# phase 3 in fitzpatrick_centaur
three = (train_df["fitzpatrick_centaur"]==3).sum()
print("amount of 3 :",three)
print("percentage of 3:", three/size * 100,"%")
print("additions 3 need to match 1:", one - three)

In [ ]:
# phase 4 in fitzpatrick_centaur
four = (train_df["fitzpatrick_centaur"]==4).sum()
print("amount of 4:",four)
print("percentage of 4:", four/size * 100,"%")
print("additions 4 need to match 1:", one - four)

In [ ]:
# phase 5 in fitzpatrick_centaur
five = (train_df["fitzpatrick_centaur"]==5).sum()
print("amount of 5:",five)
print("percentage of 5:", five/size * 100,"%")
print("additions 5 need to match 1:", one - five)

In [ ]:
six = (train_df["fitzpatrick_centaur"]==6).sum()
print("amount of 6:",six)
print("percentage of 5:", six/size * 100,"%")
print("additions 5 need to match 1:", one - six)

In [ ]:
label_counts = train_df["label"].value_counts()
print(label_counts)

## 3. Data Preprocessing


This section demonstrates basic preprocessing techniques. To enhance data quality and model performance, consider incorporating more advanced preprocessing methods.

For further guidance, feel free to take a look at the [Image Preprocessing tutorial](https://colab.research.google.com/drive/1-ItNcRMbZBE6BCwPT-wD8m3YmHqwHxme?usp=sharing)  available in the 'Resources' section of this Kaggle competition.


In [ ]:
# 3. Data Preprocessing
# Encode the labels
label_encoder = LabelEncoder()
train_df['encoded_label'] = label_encoder.fit_transform(train_df['label'])

# Split the data into training and validation sets
train_data, val_data = train_test_split(train_df, test_size=0.2, random_state=42)

# Define image data generators for training and validation
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

# Define the directory paths
train_dir = '/kaggle/input/bttai-ajl-2025/train/train/'

In [ ]:
def create_generator(dataframe, directory, batch_size=32, target_size=(128, 128)):
    """
    Template function to create image generators.
    Students should complete this function to load images and labels properly.
    """
    # Fill in the correct flow_from_dataframe parameters
    generator = train_datagen.flow_from_dataframe(
        dataframe=dataframe,
        directory=directory,
        x_col='file_path',  # Use combined path
        y_col='encoded_label',
        target_size=target_size,
        batch_size=batch_size,
        class_mode='raw',
        validate_filenames=False  # Disable strict filename validation
    )
    return generator

In [ ]:
# Create generators
train_generator = create_generator(train_data, train_dir)
val_generator = create_generator(val_data, train_dir)

In [ ]:
#taking a look at some of the photos
import matplotlib.pyplot as plt
file_name = train_df['md5hash'][0:3]
file_label = train_df['label'][0:3]
file_path = []
images = []
titles = []
for i in range(len(file_name)):
    #getting path to the image
    fn = os.path.splitext(file_name[i])[0] #without jpg
    image_dir = train_dir+file_label[i]
    images.append(Image.open(os.path.join(image_dir,file_name[i]))) #extracting the image
    file_path.append(os.path.join(image_dir,file_name[i]))
    #print(images)
    title = file_label[i]
    titles.append(title)
fig, axs = plt.subplots(1, 3, figsize=(15, 5))
for i,ax in enumerate(axs):
    ax.imshow(images[i])
    ax.axis('off')  # Hide axes for better visual appeal
    ax.set_title(titles[i])
plt.tight_layout()
plt.show();
print(file_path)

In [ ]:
def rotate_image(image_path, angle = 180):
    from PIL import Image, ImageEnhance, ImageFilter
    image = Image.open(image_path)
    return image.rotate(angle)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
ax.imshow(images[1])
ax.axis('off')  # Hide axes for better visual appeal
ax.set_title(titles[1])

plt.tight_layout()
plt.show()

In [ ]:
augemented_image_1 = rotate_image(file_path[1])
fig, ax = plt.subplots(figsize=(5, 5))
ax.imshow(augemented_image_1)
ax.axis('off')  # Hide axes for better visual appeal
ax.set_title(titles[1])

plt.tight_layout()
plt.show()

In [ ]:
augemented_image_1 = images[2].filter(ImageFilter.GaussianBlur(radius=2))
fig, ax = plt.subplots(figsize=(5, 5))
ax.imshow(augemented_image_1)
ax.axis('off')  # Hide axes for better visual appeal
ax.set_title(titles[1])

plt.tight_layout()
plt.show()

In [ ]:
from PIL import Image, ImageEnhance, ImageFilter

In [ ]:
# rotate some twos so that their equal to ones
from PIL import Image, ImageEnhance, ImageFilter
import os
skin_two = train_df[train_df["fitzpatrick_centaur"]==2]
#print(skin_two)
selected_rows = skin_two.sample(n=270, random_state = 42) #randomly select 270 images
new_entries = []  # Initialize the list before appending new rows
#image_root = train_dir
for _ , row in selected_rows.iterrows():
    #print(row)
    fn = os.path.splitext(row["md5hash"])[0]
    image_dir = train_dir+ row["label"]
    original_path = os.path.join(image_dir,row["md5hash"])
    #print(original_path)
    #try:
        #img = Image.open(original_path)
    img = Image.open(original_path)
    rotate_image = img.rotate(180)
    #rotate_image = rotate_image(original_path) #rotating image by 180 degree
    output_dir = "augmented_images"
    os.makedirs(output_dir, exist_ok = True)
    new_md5hash = row['md5hash'].split('.')[0]+ "_rot180.jpg"
    output_path = os.path.join(output_dir, new_md5hash)
    rotate_image.save(output_path, format="JPEG")  # Specify format explicitly

    #adding to overall dataframe
    new_row = row.copy()
    new_row['md5hash'] = new_md5hash
    new_entries.append(new_row)
new_entries_df = pd.DataFrame(new_entries)
train_df = pd.concat([train_df, new_entries_df], ignore_index=True)

In [ ]:
two = (train_df["fitzpatrick_centaur"]==2).sum()
print("amount of 2:",two)

In [ ]:
from PIL import Image, ImageEnhance, ImageFilter
import os
skin_three = train_df[train_df["fitzpatrick_centaur"]==3]
selected_rows = skin_three.sample(n=255, random_state = 42) #randomly select 255 images to rotat 90 and 180
new_entries = []  # Initialize the list before appending new rows
for _ , row in selected_rows.iterrows():
    #print(row)
    fn = os.path.splitext(row["md5hash"])[0]
    image_dir = train_dir+ row["label"]
    original_path = os.path.join(image_dir,row["md5hash"])
    img = Image.open(original_path)
    rotate_image_180  = img.rotate(180)
    rotate_image_90 = img.rotate(90)
    output_dir = "augmented_images"
    os.makedirs(output_dir, exist_ok = True)
    new_md5hash_180 = row['md5hash'].split('.')[0]+ "_rot180.jpg"
    new_md5hash_90 = row['md5hash'].split('.')[0]+ "_rot90.jpg"
    output_path_180 = os.path.join(output_dir, new_md5hash_180)
    output_path_90 = os.path.join(output_dir, new_md5hash_90)
    rotate_image_180.save(output_path_180, format="JPEG")  # Specify format explicitly
    rotate_image_90.save(output_path_90, format="JPEG")  # Specify format explicitly

    #adding to overall dataframe
    new_row = row.copy()
    new_row['md5hash'] = new_md5hash_180
    new_entries.append(new_row)
    new_row['md5hash'] = new_md5hash_90
    new_entries.append(new_row)
new_entries_df = pd.DataFrame(new_entries)
train_df = pd.concat([train_df, new_entries_df], ignore_index=True)

In [ ]:
three = (train_df["fitzpatrick_centaur"]==3).sum()
print("amount of 3 :",three)

In [ ]:
skin_four = train_df[train_df["fitzpatrick_centaur"] == 4]
selected_rows = skin_four.sample(n=253, random_state=42)  # Randomly select 255 images to rotate 90°, 180°, and 270°
new_entries = []  # Initialize the list before appending new rows

for _, row in selected_rows.iterrows():
    # Get file paths
    fn = os.path.splitext(row["md5hash"])[0]
    image_dir = train_dir + row["label"]
    original_path = os.path.join(image_dir, row["md5hash"])
    img = Image.open(original_path)

    # Apply rotations
    rotate_image_180 = img.rotate(180)
    rotate_image_90 = img.rotate(90)
    rotate_image_270 = img.rotate(270)  # Adding 270° rotation

    # Set output directory
    output_dir = "augmented_images"
    os.makedirs(output_dir, exist_ok=True)

    # Save rotated images with new filenames
    new_md5hash_180 = row['md5hash'].split('.')[0] + "_rot180.jpg"
    new_md5hash_90 = row['md5hash'].split('.')[0] + "_rot90.jpg"
    new_md5hash_270 = row['md5hash'].split('.')[0] + "_rot270.jpg"  # New filename for 270° rotation

    output_path_180 = os.path.join(output_dir, new_md5hash_180)
    output_path_90 = os.path.join(output_dir, new_md5hash_90)
    output_path_270 = os.path.join(output_dir, new_md5hash_270)  # Output path for 270° image

    rotate_image_180.save(output_path_180, format="JPEG")  # Specify format explicitly
    rotate_image_90.save(output_path_90, format="JPEG")  # Specify format explicitly
    rotate_image_270.save(output_path_270, format="JPEG")  # Save the 270° rotated image

    # Add new entries to overall dataframe
    new_row = row.copy()
    new_row['md5hash'] = new_md5hash_180
    new_entries.append(new_row)

    new_row['md5hash'] = new_md5hash_90
    new_entries.append(new_row)

    new_row['md5hash'] = new_md5hash_270  # Add 270° rotated image
    new_entries.append(new_row)

# Convert the new entries to a DataFrame and append it to the original DataFrame
new_entries_df = pd.DataFrame(new_entries)
train_df = pd.concat([train_df, new_entries_df], ignore_index=True)

In [ ]:
four = (train_df["fitzpatrick_centaur"]==4).sum()
print("amount of 4:",four)

In [ ]:
skin_five = train_df[train_df["fitzpatrick_centaur"] == 5]
selected_rows = skin_five.sample(n=147, random_state=42)
new_entries = []  # Initialize the list before appending new rows

shift_x = 20  # Shift 20 pixels horizontally
shift_y = 20  # Shift 20 pixels vertically

for _, row in selected_rows.iterrows():
    fn = os.path.splitext(row["md5hash"])[0]
    image_dir = os.path.join(train_dir, row["label"])
    original_path = os.path.join(image_dir, row["md5hash"])

    img = Image.open(original_path)

    # Rotation by 90, 180, 270
    rotate_image_90 = img.rotate(90)
    rotate_image_180 = img.rotate(180)
    rotate_image_270 = img.rotate(270)

    # Apply Gaussian Blur
    blurred_image = img.filter(ImageFilter.GaussianBlur(radius=2))  # Apply blur with a radius of 2

    # Apply shift (translation)
    shifted_image = img.transform(img.size, Image.AFFINE, (1, 0, shift_x, 0, 1, shift_y))  # Shift image by (shift_x, shift_y)

    # Output directory
    output_dir = "augmented_images"
    os.makedirs(output_dir, exist_ok=True)

    # Saving the rotated, shifted, and blurred images
    new_md5hash_90 = fn + "_rot90.jpg"
    new_md5hash_180 = fn + "_rot180.jpg"
    new_md5hash_270 = fn + "_rot270.jpg"
    new_md5hash_blur = fn + "_blur.jpg"
    new_md5hash_shift = fn + "_shift.jpg"

    output_path_90 = os.path.join(output_dir, new_md5hash_90)
    output_path_180 = os.path.join(output_dir, new_md5hash_180)
    output_path_270 = os.path.join(output_dir, new_md5hash_270)
    output_path_blur = os.path.join(output_dir, new_md5hash_blur)
    output_path_shift = os.path.join(output_dir, new_md5hash_shift)

    rotate_image_90.save(output_path_90, format="JPEG")
    rotate_image_180.save(output_path_180, format="JPEG")
    rotate_image_270.save(output_path_270, format="JPEG")
    blurred_image.save(output_path_blur, format="JPEG")
    shifted_image.save(output_path_shift, format="JPEG")

    # Adding to the overall dataframe
    new_row_90 = row.copy()
    new_row_90['md5hash'] = new_md5hash_90
    new_entries.append(new_row_90)

    new_row_180 = row.copy()
    new_row_180['md5hash'] = new_md5hash_180
    new_entries.append(new_row_180)

    new_row_270 = row.copy()
    new_row_270['md5hash'] = new_md5hash_270
    new_entries.append(new_row_270)

    new_row_blur = row.copy()
    new_row_blur['md5hash'] = new_md5hash_blur
    new_entries.append(new_row_blur)

    new_row_shift = row.copy()
    new_row_shift['md5hash'] = new_md5hash_shift
    new_entries.append(new_row_shift)

# Add the new augmented entries to the dataframe
new_entries_df = pd.DataFrame(new_entries)
train_df = pd.concat([train_df, new_entries_df], ignore_index=True)

In [ ]:
five = (train_df["fitzpatrick_centaur"]==5).sum()
print("amount of 5:",five)

In [ ]:
skin_six = train_df[train_df["fitzpatrick_centaur"] == 6]
selected_rows = skin_six.sample(n=80, random_state=42)  # Randomly select 80 images
new_entries = []  # Initialize the list before appending new rows

# Define shift values (you can adjust this as needed)
shift_x = 20  # Shift 20 pixels horizontally
shift_y = 20  # Shift 20 pixels vertically

# Define the number of augmentations needed per image
augmentations_per_image = 12  # Average number of augmentations (since we need to get 1000 images)

for _, row in selected_rows.iterrows():
    fn = os.path.splitext(row["md5hash"])[0]
    image_dir = os.path.join(train_dir, row["label"])
    original_path = os.path.join(image_dir, row["md5hash"])

    img = Image.open(original_path)

    # Rotation by 90, 180, 270
    rotate_image_90 = img.rotate(90)
    rotate_image_180 = img.rotate(180)
    rotate_image_270 = img.rotate(270)

    # Save rotated images first
    new_md5hash_90_rot = fn + f"_rot90.jpg"
    new_md5hash_180_rot = fn + f"_rot180.jpg"
    new_md5hash_270_rot = fn + f"_rot270.jpg"

    output_dir = "augmented_images"
    os.makedirs(output_dir, exist_ok=True)

    output_path_90_rot = os.path.join(output_dir, new_md5hash_90_rot)
    output_path_180_rot = os.path.join(output_dir, new_md5hash_180_rot)
    output_path_270_rot = os.path.join(output_dir, new_md5hash_270_rot)

    rotate_image_90.save(output_path_90_rot, format="JPEG")
    rotate_image_180.save(output_path_180_rot, format="JPEG")
    rotate_image_270.save(output_path_270_rot, format="JPEG")

    # Now apply Gaussian Blur to each rotated image
    blurred_image_90 = rotate_image_90.filter(ImageFilter.GaussianBlur(radius=2))
    blurred_image_180 = rotate_image_180.filter(ImageFilter.GaussianBlur(radius=2))
    blurred_image_270 = rotate_image_270.filter(ImageFilter.GaussianBlur(radius=2))

    # Save rotated + blurred images
    new_md5hash_90_blur = fn + f"_rot90_blur.jpg"
    new_md5hash_180_blur = fn + f"_rot180_blur.jpg"
    new_md5hash_270_blur = fn + f"_rot270_blur.jpg"

    output_path_90_blur = os.path.join(output_dir, new_md5hash_90_blur)
    output_path_180_blur = os.path.join(output_dir, new_md5hash_180_blur)
    output_path_270_blur = os.path.join(output_dir, new_md5hash_270_blur)

    blurred_image_90.save(output_path_90_blur, format="JPEG")
    blurred_image_180.save(output_path_180_blur, format="JPEG")
    blurred_image_270.save(output_path_270_blur, format="JPEG")

    # Now apply shift (translation) to each blurred image
    shifted_blurred_image_90 = blurred_image_90.transform(blurred_image_90.size, Image.AFFINE, (1, 0, shift_x, 0, 1, shift_y))
    shifted_blurred_image_180 = blurred_image_180.transform(blurred_image_180.size, Image.AFFINE, (1, 0, shift_x, 0, 1, shift_y))
    shifted_blurred_image_270 = blurred_image_270.transform(blurred_image_270.size, Image.AFFINE, (1, 0, shift_x, 0, 1, shift_y))

    # Save rotated + blurred + shifted images
    new_md5hash_90_blur_shift = fn + f"_rot90_blur_shift.jpg"
    new_md5hash_180_blur_shift = fn + f"_rot180_blur_shift.jpg"
    new_md5hash_270_blur_shift = fn + f"_rot270_blur_shift.jpg"

    output_path_90_blur_shift = os.path.join(output_dir, new_md5hash_90_blur_shift)
    output_path_180_blur_shift = os.path.join(output_dir, new_md5hash_180_blur_shift)
    output_path_270_blur_shift = os.path.join(output_dir, new_md5hash_270_blur_shift)

    shifted_blurred_image_90.save(output_path_90_blur_shift, format="JPEG")
    shifted_blurred_image_180.save(output_path_180_blur_shift, format="JPEG")
    shifted_blurred_image_270.save(output_path_270_blur_shift, format="JPEG")

    # Adding to the overall dataframe
    new_row_90_rot = row.copy()
    new_row_90_rot['md5hash'] = new_md5hash_90_rot
    new_entries.append(new_row_90_rot)

    new_row_180_rot = row.copy()
    new_row_180_rot['md5hash'] = new_md5hash_180_rot
    new_entries.append(new_row_180_rot)

    new_row_270_rot = row.copy()
    new_row_270_rot['md5hash'] = new_md5hash_270_rot
    new_entries.append(new_row_270_rot)

    new_row_90_blur = row.copy()
    new_row_90_blur['md5hash'] = new_md5hash_90_blur
    new_entries.append(new_row_90_blur)

    new_row_180_blur = row.copy()
    new_row_180_blur['md5hash'] = new_md5hash_180_blur
    new_entries.append(new_row_180_blur)

    new_row_270_blur = row.copy()
    new_row_270_blur['md5hash'] = new_md5hash_270_blur
    new_entries.append(new_row_270_blur)

    new_row_90_blur_shift = row.copy()
    new_row_90_blur_shift['md5hash'] = new_md5hash_90_blur_shift
    new_entries.append(new_row_90_blur_shift)

    new_row_180_blur_shift = row.copy()
    new_row_180_blur_shift['md5hash'] = new_md5hash_180_blur_shift
    new_entries.append(new_row_180_blur_shift)

    new_row_270_blur_shift = row.copy()
    new_row_270_blur_shift['md5hash'] = new_md5hash_270_blur_shift
    new_entries.append(new_row_270_blur_shift)

# Add the new augmented entries to the dataframe
new_entries_df = pd.DataFrame(new_entries)
train_df = pd.concat([train_df, new_entries_df], ignore_index=True)

In [ ]:
six = (train_df["fitzpatrick_centaur"]== 6).sum()
print("amount of 6:",six) #around the same amount but slightlty less

## 4. Build the model


In [ ]:
#Pretrained model DermNet
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Freeze the base model
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# layers for fine-tuning
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(train_df['label'].nunique(), activation='softmax')(x)

In [ ]:
# model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()

In [ ]:
# Unfreeze
base_model.trainable = True

for layer in base_model.layers[:-10]:  
    layer.trainable = False

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Fine-tune the model
history_fine_tune = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5,
    callbacks=[early_stopping]

In [ ]:
# IGNORE 

# Import pretrained dataset ddi_model
!git clone https://github.com/DDI-Dataset/DDI-Code.git

# Changing directory 
%cd DDI-Code

from ddi_model import load_model

pretrained_model = load_model('HAM10000')

# Freeze Base Layers
for param in pretrained_model.parameters():
    param.requires_grad = False

# Unfreeze layers 
for param in pretrained_model.fc.parameters():
    param.requires_grad = True

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(pretrained_model.parameters(), lr=0.0001)

## 5. Train the Model


In [ ]:
# TODO: Train your model here.

# Data Augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    directory='/kaggle/input/bttai-ajl-2025/train/train/',
    x_col='file_path',
    y_col='label',
    target_size=(128, 128),
    batch_size=32,
    class_mode='sparse',
    subset='training'
)

val_generator = train_datagen.flow_from_dataframe(
    train_df,
    directory='/kaggle/input/bttai-ajl-2025/train/train/',
    x_col='file_path',
    y_col='label',
    target_size=(128, 128),
    batch_size=32,
    class_mode='sparse',
    subset='validation'
)

In [ ]:
# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=[early_stopping]
)

## 6. Make Predictions on Test Data

In [ ]:
# 6. Make Predictions on Test Data
def preprocess_test_data(test_df, directory):
    """
    Template for loading and preprocessing test images.
    """
    # TODO: create a generator for the test set here.
    test_datagen = ImageDataGenerator(rescale=1./255)
    test_generator = test_datagen.flow_from_dataframe(
        test_df,
        directory=directory,
        x_col='md5hash',
        y_col=None,
        target_size=(128, 128),
        batch_size=32,
        class_mode=None,
        shuffle=False
    )
    return test_generator

# Load test data
test_dir = '/kaggle/input/bttai-ajl-2025/test/test/'
test_generator = preprocess_test_data(test_df, test_dir)

## 7. Generate Predictions

In [ ]:
# TODO
# Generate predictions based on the trained model
# Then, save the predictions into a CSV file for submission

predictions = model.predict(test_generator)
predicted_classes = predictions.argmax(axis=1)

# Create submission file
submission = pd.DataFrame({'md5hash': test_df['md5hash'], 'label': predicted_classes})
submission.to_csv('submission.csv', index=False)
print("Predictions saved to submission.csv")
